In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import matplotlib.pyplot as plt
from scipy import stats
from imblearn.over_sampling import SMOTE
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier


import joblib

In [ ]:
from zipfile import ZipFile
file_name = "pickles.zip"

with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print('Done')

In [ ]:
imp = joblib.load("pickles/imputer.pkl")
scaler = joblib.load("pickles/scaler.pkl")
bestclf = joblib.load("pickles/model.pkl")

In [ ]:
test = pd.read_csv("test_set.csv", sep = ",")

In [ ]:
asia = ["Philippines", "India", "South", "China", "Vietnam", "Japan", "Taiwan", "Iran", "Thailand", "Cambodia", "Laos", "Hong"]
europe = ["Germany", "England", "Italy", "Poland", "Portugal", "Greece", "France", "Ireland", "Yugoslavia", "Hungary", "Scotland", "Netherlands"]
north_america = ["USA", "Canada"]
central_america = ["Mexico", "Puerto-Rico", "El-Salvador", "Cuba", "Jamaica", "Dominican-Republic", "Guatemala", "Haiti", "Nicaragua", "Caribbean", "Honduras"]
south_america = ["Columbia", "Peru", "Ecuador"]

for i in range(len(test)):
    if test.loc[i, "F13"] in asia:
        test.F13.replace(test.loc[i, "F13"], "asia", inplace = True)
    elif test.loc[i, "F13"] in europe:
        test.F13.replace(test.loc[i, "F13"], "europe", inplace = True)
    elif test.loc[i, "F13"] in north_america:
        test.F13.replace(test.loc[i, "F13"], "north_america", inplace = True)
    elif test.loc[i, "F13"] in central_america:
        test.F13.replace(test.loc[i, "F13"], "central_america", inplace = True)
    elif test.loc[i, "F13"] in south_america:
        test.F13.replace(test.loc[i, "F13"], "south_america", inplace = True)       

In [ ]:
colF1 = ['K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8']
colF3 = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R16']
colF5 = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7']
colF6 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14']
colF7 = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']
colF8 = ['caucasian', 'black', 'asian', 'american', 'other']
colF9 = ['Male', 'Female']
colF13 = ['asia', 'europe', 'north_america', 'central_america', 'south_america']

test.F1.replace(colF1, [0, 1, 2, 3, 4, 5, 6, 7], inplace = True)
test.F3.replace(colF3, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], inplace = True)
test.F5.replace(colF5, [0, 1, 2, 3, 4, 5, 6], inplace = True)
test.F6.replace(colF6, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], inplace = True)
test.F7.replace(colF7, [0, 1, 2, 3, 4, 5], inplace = True)
test.F8.replace(colF8, [0, 1, 2, 3, 4], inplace = True)
test.F9.replace(colF9, [0, 1], inplace = True)
test.F13.replace(colF13, [0, 1, 2, 3, 4], inplace = True)

In [ ]:
x_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Imputazione dei valori NaN tramite SimpleImputer, con la strategia che sostituisce i valori mancanti con la moda dei valori della feature
# print("SimpleImputer")
# imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
# x_test_notnan = imp.transform(x_test)
# print(x_test_notnan[pd.isna(x_test[:,0]),0][0:10])

x_test[:] = imp.transform(x_test)

In [ ]:
# Creazione DataFrame da insiemi di train e test dopo l'imputazione dei valori mancanti 
colF = ['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']

test_set_df = pd.DataFrame(x_test_notnan, columns = colF)

In [ ]:
# Codifica OneHot contemporaneamente su training_set_df e test_set_df
OneHotTestF1 = pd.get_dummies(test_set_df.F1)
OneHotTestF3 = pd.get_dummies(test_set_df.F3)
OneHotTestF5 = pd.get_dummies(test_set_df.F5)
OneHotTestF6 = pd.get_dummies(test_set_df.F6)
OneHotTestF7 = pd.get_dummies(test_set_df.F7)
OneHotTestF8 = pd.get_dummies(test_set_df.F8)
OneHotTestF9 = pd.get_dummies(test_set_df.F9)
OneHotTestF13 = pd.get_dummies(test_set_df.F13)

for i in range (0, len(colF1)):
    OneHotTestF1 = OneHotTestF1.rename(columns = {i: colF1[i]})
for i in range (0, len(colF3)):
    OneHotTestF3 = OneHotTestF3.rename(columns = {i: colF3[i]})
for i in range (0, len(colF5)):
    OneHotTestF5 = OneHotTestF5.rename(columns = {i: colF5[i]})
for i in range (0, len(colF6)):
    OneHotTestF6 = OneHotTestF6.rename(columns = {i: colF6[i]})
for i in range (0, len(colF7)):
    OneHotTestF7 = OneHotTestF7.rename(columns = {i: colF7[i]})
for i in range (0, len(colF8)):
    OneHotTestF8 = OneHotTestF8.rename(columns = {i: colF8[i]})
for i in range (0, len(colF9)):
    OneHotTestF9 = OneHotTestF9.rename(columns = {i: colF9[i]})
for i in range (0, len(colF13)):
    OneHotTestF13 = OneHotTestF13.rename(columns = {i: colF13[i]})

In [ ]:
# Creazione newdf e newdftest con la sostituzione delle colonne generate con la codifica
test_set_df = pd.concat([test_set_df, OneHotTestF1, OneHotTestF3, OneHotTestF5, OneHotTestF6, OneHotTestF7, OneHotTestF8, OneHotTestF9, OneHotTestF13], axis = 1)
 
test_set_df = test_set_df.drop(["F1", "F3", "F5", "F6", "F7", "F8", "F9", "F13"], axis = 1)

In [ ]:
x_test[:] = scaler.transform(x_test)

In [ ]:
y_pred = bestclf.predict(x_test)

copiare da ml la parte di confusion matrix